In [1]:
import os

import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

import config
from retina_face.retina_face import RetinaFaceModel
from src.utils import draw_box_name
from dataset_utils.mtcnn_pytorch.src.align_trans import get_reference_facial_points, warp_and_crop_face

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
retina_face = RetinaFaceModel('mobilenet', device)

RuntimeError: Error(s) in loading state_dict for MobileNetV1:
	Missing key(s) in state_dict: "stage1.0.0.weight", "stage1.0.1.weight", "stage1.0.1.bias", "stage1.0.1.running_mean", "stage1.0.1.running_var", "stage1.1.0.weight", "stage1.1.1.weight", "stage1.1.1.bias", "stage1.1.1.running_mean", "stage1.1.1.running_var", "stage1.1.3.weight", "stage1.1.4.weight", "stage1.1.4.bias", "stage1.1.4.running_mean", "stage1.1.4.running_var", "stage1.2.0.weight", "stage1.2.1.weight", "stage1.2.1.bias", "stage1.2.1.running_mean", "stage1.2.1.running_var", "stage1.2.3.weight", "stage1.2.4.weight", "stage1.2.4.bias", "stage1.2.4.running_mean", "stage1.2.4.running_var", "stage1.3.0.weight", "stage1.3.1.weight", "stage1.3.1.bias", "stage1.3.1.running_mean", "stage1.3.1.running_var", "stage1.3.3.weight", "stage1.3.4.weight", "stage1.3.4.bias", "stage1.3.4.running_mean", "stage1.3.4.running_var", "stage1.4.0.weight", "stage1.4.1.weight", "stage1.4.1.bias", "stage1.4.1.running_mean", "stage1.4.1.running_v

In [ ]:
image_path = "input/mr_bean.jpg"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

bounding_boxes, faces, landmarks = retina_face.detect(image)
print('number of detected faces: ', len(faces))

for i, bbox in enumerate(bounding_boxes):
    cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (255, 0, 0), 2)
    for landmark in landmarks[i]:
        cv2.circle(image, (landmark[0], landmark[1]), 8, (0, 255, 0), -1)

In [ ]:
plt.imshow(image)

In [ ]:
landmark = landmarks[i]
reference = get_reference_facial_points(default_square=True)
warped_face = warp_and_crop_face(np.array(image), landmark, reference, crop_size=(112, 112))
plt.imshow(warped_face)

In [ ]:
reference = np.array([
    [30.29459953,  51.69630051],
    [65.53179932,  51.50139999],
    [48.02519989,  71.73660278],
    [33.54930115,  92.3655014],
    [62.72990036,  92.20410156]
])

landmark = landmarks[i]
landmark = np.array(landmark)

In [ ]:
from skimage import transform as trans

tform = trans.SimilarityTransform()
tform.estimate(landmark, reference)
M = tform.params[0:2,:]

In [ ]:
wrapped = cv2.warpAffine(image ,M,(96,112), borderValue = 0.0)
plt.imshow(wrapped)